In [30]:
import requests
import json
from pathlib import Path
import re
import os
import psycopg2

# 1. Define Functions & Parameters

In [2]:
OLLAMA_API_URL = "http://localhost:11434/api/chat"

In [3]:
def initial_setup():
    instruction = 'You are a friendly and helpful data analyst. The user will ask you question about data analysis, be prepared.'
    chat= [{'role':'system',
            'content':instruction}]
    payload = {
    "model":"deepseek-r1:8b",
    "messages":chat,
    "stream":False,
    "temperature": 0.0
}
    response = requests.post(OLLAMA_API_URL,json=payload).json()

    chat.append({'role':'assistant',
                        'content':response['message']['content']})
    return chat

In [4]:
def generate_sql_query(natural_language):
    # Get the current directory of the script
    script_dir = Path.cwd()

    # Navigate to the parent directory and then into the Context folder
    context_dir = script_dir.parent / "Context"

    # Construct the full path to the file
    file_path = context_dir / "MASTER_FUNDING.txt"

    # Open context file in read mode
    with open(file_path, "r") as file:
        # Read the entire file content
        content = file.read()

    prompt = f"""
You are an expert SQL generator. Translate the following natural language request into a valid SQL query. 
Make sure the query is properly formatted and is tailored for our database schema.
The table in the database is master_funding, it uses the following schema:
{content}

Request: {natural_language}

SQL Query:
"""
    payload = {
        "model": "deepseek-r1:8b",  # if needed, or remove if the API defaults to it
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.0,  # Lower temperature can help ensure deterministic output
        "stream":False
    }
    response = requests.post(OLLAMA_API_URL, json=payload)
    response_data = response.json()
    
    # The structure may vary based on the API; assume generated text is in this field:
    sql_query = response_data.get("generated_text", "").strip()
    return sql_query

In [5]:
def generate_insight(natural_language, sql_query, query_result):
    # Format the query result in a readable way (e.g., JSON)
    formatted_result = json.dumps(query_result, indent=2)
    
    prompt = f"""
You are a data analyst. You first received the following natural language request:
"{natural_language}"

Based on that, an SQL query was generated and executed:
{sql_query}

The query returned the following result:
{formatted_result}

Please provide a detailed explanation of the insights, trends, or key points that can be derived from these results.
"""
    payload = {
        "model": "deepseek-r1:8b",
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.7,
        "max_tokens": 300
    }
    response = requests.post(OLLAMA_API_URL, json=payload)
    response_data = response.json()
    insight = response_data.get("generated_text", "").strip()
    return insight


In [22]:
def query_database(sql_query):
    API_URL = "http://127.0.0.1:8000/query"
    response = requests.post(API_URL, json={"query": sql_query})
    return response.json()

# Testing

In [6]:
chat_hist = initial_setup()

In [7]:
chat_hist

[{'role': 'system',
  'content': 'You are a friendly and helpful data analyst. The user will ask you question about data analysis, be prepared.'},
 {'role': 'assistant',
  'content': "<think>\nOkay, so the user wants to understand how to approach data analysis in their project. Let me break down what they might need.\n\nFirst, I should figure out the main goal of their project. Is it descriptive analytics, which means summarizing data, or something more advanced like predictive analytics? Knowing this will help determine the tools and techniques to use.\n\nNext, they'll probably need a clear dataset. What variables are involved? Are there any missing values or outliers that could affect the analysis? Cleaning the data might be necessary at this stage.\n\nThey might also require transforming the data to make it more usable. Maybe they need to create new features or handle time series data. I should consider what kind of transformations would add value.\n\nChoosing the right tools is imp

In [8]:
natural_language = "What month has the highest amount of funding?"

# Get the current directory of the script
script_dir = Path.cwd()

# Navigate to the parent directory and then into the Context folder
context_dir = script_dir.parent / "Context"

# Construct the full path to the file
file_path = context_dir / "MASTER_FUNDING.txt"

# Open context file in read mode
with open(file_path, "r") as file:
    # Read the entire file content
    content = file.read()

prompt = f"""
You are an expert SQL generator. Translate the following natural language request into a valid SQL query. 
Make sure the query is properly formatted and is tailored for our database schema.
The table in the database is master_funding, it uses the following schema:
{content}

Request: {natural_language}

SQL Query:
"""

chat_hist.append({"role":"user","content":prompt})
payload = {
    "model": "deepseek-r1:8b",  # if needed, or remove if the API defaults to it
    "messages": chat_hist,
    "temperature": 0.0,  # Lower temperature can help ensure deterministic output
    "stream":False
}
response = requests.post(OLLAMA_API_URL, json=payload).json()

In [9]:
reply = response['message']['content']
chat_hist.append({'role':'assistant',
                  'content':reply})

In [18]:
reply

'<think>\nOkay, so I need to figure out which month has the highest amount of funding. The data is stored in a table called master_funding, and I have access to various fields related to each account\'s funding details.\n\nFirst, I should identify the relevant columns for this query. The user is asking about the month with the highest funding, so I need a column that represents the month, like BASE_YM, which is set as YYYYMM. That seems perfect because it captures the year and month in a four-digit format.\n\nNext, I need to find out the amount of funding for each month. The field BASE_AMT_FIX appears to hold the end-of-day funding amount, so that\'s the measure we want to maximize.\n\nI should group the data by BASE_YM to aggregate the funding amounts per month. Using the GROUP BY clause in SQL would help with this.\n\nOnce grouped, I can calculate the maximum value for each month using MAX(BASE_AMT_FIX). This will give me the highest funding amount for each specific month across all 

In [19]:
# Get SQL query
pattern = r"```sql\n(.*?)\n```"

# Search for the pattern
match = re.search(pattern, reply, re.DOTALL)

# Extracted SQL query
sql_query = match.group(1) if match else None

In [21]:
print(sql_query)

SELECT DISTINCT 
    BASE_YM AS "Month",
    MAX(BASE_AMT_FIX) AS "Highest Funding Amount"
FROM master_funding
GROUP BY BASE_YM
ORDER BY "Highest Funding Amount" DESC;


In [32]:
# try using the api
query_database(sql_query)

{'data': [{'month': 202501, 'highest funding amount': 535000000000.0},
  {'month': 202411, 'highest funding amount': 153809364652.52},
  {'month': 202412, 'highest funding amount': 150000000000.0},
  {'month': 202405, 'highest funding amount': 140610631177.91},
  {'month': 202407, 'highest funding amount': 102000407000.0},
  {'month': 202403, 'highest funding amount': 100000000000.0},
  {'month': 202408, 'highest funding amount': 82684250000.0},
  {'month': 202404, 'highest funding amount': 65115312742.8},
  {'month': 202410, 'highest funding amount': 65000000000.0},
  {'month': 202406, 'highest funding amount': 56000000000.0},
  {'month': 202212, 'highest funding amount': 50361419319.0},
  {'month': 202310, 'highest funding amount': 50000000000.0},
  {'month': 202401, 'highest funding amount': 50000000000.0},
  {'month': 202308, 'highest funding amount': 46609845490.1764},
  {'month': 202309, 'highest funding amount': 34260353348.16},
  {'month': 202409, 'highest funding amount': 2700

In [34]:
if sql_query:
    print("Executing SQL query")
    result = query_database(sql_query)
    print("Query Result:", result)
else:
    print("No SQL query found in response.")

Executing SQL query: SELECT DISTINCT 
    BASE_YM AS "Month",
    MAX(BASE_AMT_FIX) AS "Highest Funding Amount"
FROM master_funding
GROUP BY BASE_YM
ORDER BY "Highest Funding Amount" DESC;
Query Result: {'data': [{'month': 202501, 'highest funding amount': 535000000000.0}, {'month': 202411, 'highest funding amount': 153809364652.52}, {'month': 202412, 'highest funding amount': 150000000000.0}, {'month': 202405, 'highest funding amount': 140610631177.91}, {'month': 202407, 'highest funding amount': 102000407000.0}, {'month': 202403, 'highest funding amount': 100000000000.0}, {'month': 202408, 'highest funding amount': 82684250000.0}, {'month': 202404, 'highest funding amount': 65115312742.8}, {'month': 202410, 'highest funding amount': 65000000000.0}, {'month': 202406, 'highest funding amount': 56000000000.0}, {'month': 202212, 'highest funding amount': 50361419319.0}, {'month': 202310, 'highest funding amount': 50000000000.0}, {'month': 202401, 'highest funding amount': 50000000000.0},

In [35]:
formatted_result = json.dumps(result, indent=2)

In [36]:
# generate insight
prompt = f"""
You are a data analyst. You first received the following natural language request:
"{natural_language}"

Based on that, an SQL query was generated and executed:
{sql_query}

The query returned the following result:
{formatted_result}

Please provide a detailed explanation of the insights, trends, or key points that can be derived from these results.
"""
chat_hist.append({"role":"user","content":prompt})

payload = {
    "model": "deepseek-r1:8b",
    "messages": chat_hist,
    "temperature": 0.7,
    "stream":False
}
response = requests.post(OLLAMA_API_URL, json=payload).json()

In [39]:
response['message']['content']

"\n\n**Insights and Key Points Derived from the Results**\n\n1. **Month with the Highest Funding:**\n   - **January 2025 (202501)** holds the title of having the highest funding amount of **$535,000,000,000.00**, significantly higher than any other month.\n\n2. **Trend Observations:**\n   - **Seasonal Variation:** The data shows a noticeable decrease in funding amounts as we move away from January 2025 towards later months, indicating a peak in funding during the beginning of the year.\n   \n3. **Key Funding Peaks:**\n   - Besides January, other months like November (202411) and December (202412) also have notably high funding amounts, suggesting that significant investments occur towards the end of the year.\n\n4. **Consistency in Mid-Year Months:**\n   - Months around mid-year, such as July (202407), August (202408), and September (202409), consistently receive substantial funding, but they are overshadowed by January's peak.\n\n5. **Year-on-Year Comparison:**\n   - Comparing across 

In [29]:
# connection info
host= 'localhost'
port= '5432'
database= 'postgres'
username= 'postgres'
password= os.environ['PGPW']

In [33]:
if 1:
    print("Hello world")

Hello world
